In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

url_list = "http://www.btbtdy.com/screen/1--2017---time-1.html"

driver = webdriver.PhantomJS('C:\\Users\\Kevin_Chiou\\MovieCrawler\\phantomjs.exe')
# get()方法會一直等到頁面被完全加載，然後才會繼續程序
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
#print (soup.prettify())
# p_list = soup.find_all('p', class_='title')

# for p in p_list:
#     print(p.contents[0].text)


# a_list = soup.find_all('a', target='_blank')
# for a in a_list:
#     print(a)
#     print("a.content is %s" % (a.contents))  

# a_list = soup.find_all('a', target='_blank')
# for a in a_list:
#     print(a)
#     print("a.content is %s" % (a.contents))  



# 打印獲取的文本內容
#print(data)

# 打印頁面標題：XXXX年最新电影_第1页-BT电影天堂
print(driver.title)

p_list = soup.find_all('p', class_='title')
f = open('MovieNames.txt', 'w', encoding = 'UTF-8')    # 也可使用指定路徑等方式，如： C:\A.txt
for p in p_list:
    f.write(p.contents[0].text + '\n')

f.close()    
    
driver.quit()

# http://www.btbtdy.com/ herf=btdy/dy11737.html

2017年最新电影_第1页-BT电影天堂


## 讀取txt檔案

In [21]:
f = open("C:\\Users\\Kevin_Chiou\\MovieCrawler\\MovieNames.txt","r", encoding = 'UTF-8')
for line in f:
    print(line[:-1])
#In [13]:l = list(range(10))
#In [13]: l
#Out[13]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#In [16]: l[:-1]
#Out[16]: [0, 1, 2, 3, 4, 5, 6, 7, 8]

f.close() 


暴雪将至
阴阳路之常在你左右
大嫂
缝纫机乐队
百日孤独 Sum1
母亲！
兄弟，别闹！
冒牌监护人之寻宝闹翻天
这就是命
镇魂法师
漫画少年的遐想世界
血战湘江
蛰伏
美玉
当哈利遇到莎迦
超重警官
厕所：一个爱的故事
悍妻理论
西尔玛
维京：王者之战
鲨海
昼颜 电影版
夺金四剑客
敦刻尔克
烟花
寻梦环游记
底特律暴乱
厄运
迷失天堂2
肥皂泡
从未在此
方法派
末日重启
异形：汇合
爱情邮局
杀人狂


## 爬Imdb頁面

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

url_list = "http://www.imdb.com/title/tt5556068/"

driver = webdriver.PhantomJS('C:\\Users\\Kevin_Chiou\\MovieCrawler\\phantomjs.exe')
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

d_list = soup.find_all('div', class_='ratingValue')

print(d_list)
# for d in d_list:
#     print(d.descendants)

driver.quit()

## 爬Imdb頁面

## 爬Imdb頁面

## 整理code如下


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

url_list = "https://movie.douban.com/"

driver = webdriver.PhantomJS('C:\\Users\\Kevin_Chiou\\MovieCrawler\\phantomjs.exe')
driver.get(url_list)

# 把selenium的webdriver调用page_source函数在传入BeautifulSoup中，就可以用BeautifulSoup解析网页了
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

#找豆瓣電影的search bar
inputSearch = driver.find_element_by_name("search_text")

#這邊輸入你要的中文電影名稱 EX: ChineseMovieName = u"挖崊老ㄙ"
ChineseMovieName = u"天才槍手"

inputSearch.send_keys(ChineseMovieName)
inputSearch.submit()
time.sleep(2)

#印出找到的電影名稱跟豆瓣評分
searchOutput = driver.find_element_by_class_name("title")
print(searchOutput.text)
searchHerf = searchOutput.find_element_by_css_selector('a').get_attribute('href')
print(searchHerf)
rating_num = driver.find_element_by_class_name("rating_nums")
print("豆瓣 : "+rating_num.text)

#點進去該電影後找Imdb的連結
driver.get(searchHerf)
# get()方法會一直等到頁面被完全加載，然後才會繼續程序, 所以不需要sleep等待
#time.sleep(3)
html = driver.page_source
IMDBLink = driver.find_element_by_partial_link_text("tt").get_attribute('href')
#IMDBLink = IMDBLink.find_element_by_css_selector('a').get_attribute('href')
#//span[contains(@class, 'pl')]
#//a[contains(text(), 'tt')]/@href
#//a[contains(@href, 'imdb')]
print(IMDBLink)
driver.get(IMDBLink)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


<bound method WebElement.screenshot of <selenium.webdriver.remote.webelement.WebElement (session="a91dd440-da51-11e7-bcf9-5b89084d3eee", element=":wdc:1512543034996")>>
天才枪手 ฉลาดเกมส์โกง (2017)[可播放]
https://movie.douban.com/subject/27024903/
豆瓣 : 8.2
http://www.imdb.com/title/tt6788942


In [ ]:
ImdbRating = driver.find_element_by_class_name("ratingValue")
print("IMDB : " +ImdbRating.text)
movieName = driver.find_element_by_class_name("title_wrapper")
print(movieName.text)
imdbMovieName = driver.find_element_by_xpath("//*[@id='title-overview-widget']//h1")
print(imdbMovieName.text)
#//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/h1/text()
#"//*[@id='UserContent']//span[@itemprop='description']"


#driver.quit()

## 因為不知道如何filter掉 h1後方的span, 乾脆用soup把所有span拔掉

In [ ]:
while len(soup.find_all('span')) > 0:
    soup.span.extract()
imdbMovieName2 = soup.find('h1', itemprop="name").getText()
print(imdbMovieName2)

## 將得到的英文電影名稱丟到Rotten Tomato裡面查評價

In [ ]:
RottenTomatoURL = "https://www.rottentomatoes.com/"
RTpage = driver.get(RottenTomatoURL)


In [ ]:
print(imdbMovieName2)
RTsearchURL = "https://www.rottentomatoes.com/search/?search="
RTSearch = driver.get(RTsearchURL+imdbMovieName2
# get()方法會一直等到頁面被完全加載，然後才會繼續程序, 所以不需要sleep等待
# time.sleep(2)

In [ ]:
Score = driver.find_element_by_class_name("tmeter")
###//span[contains(@class, 'tMeterScore')]
print(Score.text)